In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
df = pd.read_csv('cumulative.csv')
print(f'Всего строк: {df.shape[0]}, столбцов: {df.shape[1]}')

Всего строк: 9564, столбцов: 50


In [4]:
SAMPLE_SIZE = 500     # возьмём 500 случайных записей
df = df.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)

TARGET = 'koi_disposition'   # 3 класса: CONFIRMED / FALSE POSITIVE / CANDIDATE
y = df[TARGET]

In [5]:
drop_cols = ['rowid', 'kepid', 'kepoi_name', 'kepler_name',
             TARGET, 'koi_pdisposition', 'koi_tce_delivname']
X = df.drop(columns=[col for col in drop_cols if col in df.columns])

numeric_cols = X.select_dtypes(include=[np.number]).columns
X = X[numeric_cols]

print(f'Используем {X.shape[1]} числовых признаков.')


Используем 43 числовых признаков.


In [6]:
le = LabelEncoder()
y_enc = le.fit_transform(y)
print('Классы:', dict(zip(le.classes_, le.transform(le.classes_))))


Классы: {'CANDIDATE': np.int64(0), 'CONFIRMED': np.int64(1), 'FALSE POSITIVE': np.int64(2)}


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)


In [8]:
logreg_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=500, multi_class='multinomial',
                               solver='lbfgs'))
])

logreg_pipeline.fit(X_train, y_train)
y_pred_lr = logreg_pipeline.predict(X_test)


/Users/vladflorinskij/Documents/6sem/ML_6sem/myenv/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/vladflorinskij/Documents/6sem/ML_6sem/myenv/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [9]:
gb_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('clf', HistGradientBoostingClassifier(random_state=42))
])

gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_test)


/Users/vladflorinskij/Documents/6sem/ML_6sem/myenv/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/vladflorinskij/Documents/6sem/ML_6sem/myenv/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_teq_err1' 'koi_teq_err2']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [10]:
def evaluate(y_true, y_pred, name='Модель'):
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average='macro')
    print(f'\n{name}')
    print('-'*40)
    print(f'Accuracy : {acc:7.3f}')
    print(f'Macro-F1 : {f1:7.3f}')
    return acc, f1

acc_lr, f1_lr = evaluate(y_test, y_pred_lr, 'Логистическая регрессия')
acc_gb, f1_gb = evaluate(y_test, y_pred_gb, 'Градиентный бустинг')



Логистическая регрессия
----------------------------------------
Accuracy :   0.860
Macro-F1 :   0.814

Градиентный бустинг
----------------------------------------
Accuracy :   0.880
Macro-F1 :   0.839


In [11]:
print('\nClassification report (LogReg):')
print(classification_report(y_test, y_pred_lr, target_names=le.classes_))

print('\nClassification report (GradBoost):')
print(classification_report(y_test, y_pred_gb, target_names=le.classes_))



Classification report (LogReg):
                precision    recall  f1-score   support

     CANDIDATE       0.74      0.61      0.67        23
     CONFIRMED       0.78      0.88      0.82        24
FALSE POSITIVE       0.94      0.96      0.95        53

      accuracy                           0.86       100
     macro avg       0.82      0.82      0.81       100
  weighted avg       0.86      0.86      0.86       100


Classification report (GradBoost):
                precision    recall  f1-score   support

     CANDIDATE       0.76      0.70      0.73        23
     CONFIRMED       0.80      0.83      0.82        24
FALSE POSITIVE       0.96      0.98      0.97        53

      accuracy                           0.88       100
     macro avg       0.84      0.84      0.84       100
  weighted avg       0.88      0.88      0.88       100



In [12]:
print('\n' + '='*60)
print('ВЫВОДЫ:')
print('1. Accuracy показывает долю верных предсказаний, но чувствительна к перекосу классов;')
print('   Macro-F1 усредняет F1 по классам, сглаживая дисбаланс и оценивая качество')
print('   одновременно по полноте и точности для каждой категории.')
better = 'Градиентный бустинг' if f1_gb > f1_lr else 'Логистическая регрессия'
print(f'2. По Macro-F1 лучшей оказалась модель: **{better}** '
      f'({max(f1_lr, f1_gb):.3f} против {min(f1_lr, f1_gb):.3f}).')
print('3. Линейная модель быстрее обучается, но бустинг лучше '
      'уловил нелинейные зависимости в данных.')
print('='*60)


ВЫВОДЫ:
1. Accuracy показывает долю верных предсказаний, но чувствительна к перекосу классов;
   Macro-F1 усредняет F1 по классам, сглаживая дисбаланс и оценивая качество
   одновременно по полноте и точности для каждой категории.
2. По Macro-F1 лучшей оказалась модель: **Градиентный бустинг** (0.839 против 0.814).
3. Линейная модель быстрее обучается, но бустинг лучше уловил нелинейные зависимости в данных.
